# Plan

1. Run local LLMs with Langchain
2. Local agents <-
3. Orchestrate multiple agents: CrewAI

# 2.Local agents with Langchain

#### What's an agent? 


From `Langchain`: the core idea of agents is to use a language model to _choose a sequence of actions_ to take. 
- In chains, a sequence of actions is hardcoded (in code)
- In agents, a language model determines which actions to take and in which order

Basically:
- autonomous `LLM` + `tools`

## Set LLM model

In [1]:
# %pip install --quiet google-api-python-client
# %pip install --quiet  duckduckgo-search
# %pip install --quiet  wikipedia

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain import hub

from langchain_community.llms import GPT4All
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_openai import OpenAI

from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import load_tools
from langchain.agents import Tool

In [3]:
# load api key from .env
load_dotenv('./.env', override=True)

sid = os.getenv("GOOGLE_CSE_ID")
google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# set path to local models
wizard = "./models/wizardlm-13b-v1.2.Q4_0.gguf"
mistral = "./models/mistral-7b-instruct-v0.1.Q4_0.gguf"

In [4]:
# function to set a llm model

def set_llm(llm=None, model=None):
  '''choose a platform and a model to use as the llm agent'''
  
  if llm == 'openai':
    return OpenAI(temperature=0.2, openai_api_key=openai_api_key)
  
  elif llm == 'gpt4all':
    return GPT4All(
      model=model, 
      callbacks=[StreamingStdOutCallbackHandler()], 
      verbose=True
    )
    
  elif llm == 'ollama':
    return ChatOllama(
      model=model,
      callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
    )
  
  else:
    return None
    print (f"platform/model not found: {llm}/{model}")
    
  print (f"llm is set to {llm} with model {model}")


In [5]:
## choose a models as agent: use function above

# # openai
# llm = OpenAI(temperature=0.2, openai_api_key=openai_api_key)

# # gpt4all
# local_path = ("./models/wizardlm-13b-v1.2.Q4_0.gguf")
# llm = GPT4All(
#   model=local_path, 
#   callbacks=[StreamingStdOutCallbackHandler()], 
#   verbose=True
# )

# # ollama
# llm = ChatOllama(
#   model="llama2",
#   callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
# )

In [6]:
# set the llm
llm = set_llm(llm='gpt4all', model=wizard)
# llm = set_llm(llm='ollama', model='llama2')
# llm = set_llm(llm='openai')

## Search tools

To be effective, the base model of the LLM should be trained on tasks related to the tools (code, math, APIs).

Some other toolset: maths, code, query data, Gsuite, etc.

In [7]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

## define tools to use, can build custom tools

# google search
google = GoogleSearchAPIWrapper(google_api_key=google_api_key, google_cse_id=sid)
search_gg = Tool(
    name="search google",
    description="Search Google for up to date results",
    func=google.run
)

# duckduckgo search
duck = DuckDuckGoSearchRun()
search_duck = Tool(
    name="search duckduckgo",
    description="Search DuckDuckGo for recent information",
    func=duck.run
)

# wikipedia search
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000))
search_wiki = Tool(
    name="search wikipedia",
    description="Search Wikipedia articles for grounded facts and concepts",
    func=wikipedia.run
)

## create a list of tools: need larger/better models to handle multiple tools

# tools = [search_gg, search_duck, search_wiki]
tools = [search_wiki] 

In [8]:
# get prompt template
prompt = hub.pull("hwchase17/react") # can customize prompt here

# Construct the agent: ReactAgent is the easiest one with local LLMs
agent = create_react_agent(llm, tools, prompt)

# Set agent executor: control how agents interact with tools
agent_executor = AgentExecutor(
  agent=agent, 
  tools=tools, 
  verbose=True, 
  handle_parsing_errors=True,
  max_iterations=6 # max number of calls
)

In [9]:
# tricky question
agent_executor.invoke({"input": "Which country is the smallest richest country in the world?"})



> Entering new AgentExecutor chain...
 I should search for a list of countries by GDP per capita
Action: search wikipedia
Action Input: List of countries by GDP (nominal) per capitaPage: List of countries by GDP (nominal) per capita
Summary: The figures presented here do not take into account differences in the cost of living in different countries, and the results vary greatly from one year to another based on fluctuations in the exchange rates of the country's currency. Such fluctuations change a country's ranking from one year to the next, even though they often make little or no difference to the standard of living of its population.
GDP per capita is often considered an indicator of a country's standard of living; however, this is inaccurate because GDP per capita is not a measure of personal income.
Comparisons of national income are also frequently made on the basis of purchasing power parity (PPP), to adjust for differences in the cost of living in different countries. (see L

{'input': 'Which country is the smallest richest country in the world?',
 'output': 'According to the list of countries by GDP (PPP) per capita, the smallest richest country in the world is currently Qatar with a GDP (PPP) per capita of Int$ 138,910 in 2019.'}